In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from keras.layers import Dense, Input
from keras import Sequential

Using TensorFlow backend.


In [2]:
trainDf = pd.read_csv('/import/scratch/mdl31/ctrdata/train')

In [3]:
# Don't want to onehot encode these variable sbut don't want to delete either
hourSeries = trainDf['hour']
idSeries = trainDf['id']
ySeries = trainDf['click']

In [4]:
trainDf = trainDf.drop('hour', axis = 1)
trainDf = trainDf.drop('id', axis = 1)
trainDf = trainDf.drop('click', axis = 1)


# Removing these as there are too many categories
# TODO could try to group into broad categories, e.g. take top 5 commonest categroies, and put the rest under 'other' token
trainDf = trainDf.drop('device_id', axis = 1)
trainDf = trainDf.drop('device_ip', axis = 1)
trainDf = trainDf.drop('device_model', axis = 1)
trainDf = trainDf.drop('site_id', axis = 1)
trainDf = trainDf.drop('site_domain', axis = 1)
trainDf = trainDf.drop('app_id', axis = 1)
trainDf = trainDf.drop('app_domain', axis = 1)
trainDf = trainDf.drop('C14', axis = 1)
trainDf = trainDf.drop('C17', axis = 1)
trainDf = trainDf.drop('C19', axis = 1) # only 68 values so might want
trainDf = trainDf.drop('C20', axis = 1)

In [5]:
# This cell was used to check number of categories in each diemnsion,
# in order to choose which to remove
combs = 1
for col in trainDf.columns:
    print(col)
    length = len(trainDf[col].value_counts())
    combs *= length
    print(length)

C1
7
banner_pos
7
site_category
26
app_category
36
device_type
5
device_conn_type
4
C15
8
C16
9
C18
4
C21
60


In [6]:
# one-hot encode the variables
trainDf = pd.get_dummies(trainDf, columns = trainDf.columns, drop_first=True)

In [7]:
len(trainDf.columns)

156

In [8]:
# Reintroduce the hour data
trainDf['hour'] = hourSeries
del hourSeries


In [9]:
# Create validation dataset
x_train, x_val, y_train, y_val = train_test_split(trainDf, ySeries, stratify = ySeries)
del trainDf

In [10]:
'''# Now to start building a model
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(max_iter=2, loss='hinge')
model.fit(x_train,y_train)
y_train_pred = model.pred(x_train)
accuracy_score(y_train,y_train_pred)'''

"# Now to start building a model\nfrom sklearn.linear_model import SGDClassifier\nmodel = SGDClassifier(max_iter=2, loss='hinge')\nmodel.fit(x_train,y_train)\ny_train_pred = model.pred(x_train)\naccuracy_score(y_train,y_train_pred)"

In [11]:
# Create Neural Network model
nnModel = Sequential()
nnModel.add(Dense(1024, input_shape=(len(x_train.columns),), activation='relu'))
nnModel.add(Dense(512, activation='relu'))
nnModel.add(Dense(256, activation='relu'))
nnModel.add(Dense(1,   activation='sigmoid'))
nnModel.compile(optimizer='adam', metrics=['accuracy'],loss='binary_crossentropy')
nnModel.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              161792    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 818,177
Trainable params: 818,177
Non-trainable params: 0
_________________________________________________________________


In [13]:
nnModel.fit(x_train,y_train, batch_size = 256, epochs = 2)

Epoch 1/2
30321725/30321725 [==============================] - 1088s 36us/step - loss: 13.2353 - acc: 0.1698
Epoch 2/2
30321725/30321725 [==============================] - 1076s 35us/step - loss: 13.2353 - acc: 0.1698


I ran out of time to take this any further (time limit was set to 3hrs). Model does not have better accuracy than simply guessing that the user will not click. This was the case even when trained for much larger numbers of epochs.
 Next steps I would take:
- Address class imbalance: use class weights or split data to have even classes. This should encourage better classification
- Try a different model or a more complex neural network to resolve the underfitting
- Try retaining the features that were deleted
- try hashing rather than onehot encoding the categorical data
- If i get reasonable training results, train the model with validation data supplied to know when to stop training the NN, use a checkpoint to save the model and to do early stopping
- If i get reasonable training results, then evaluate against the test data to get an estimation of accuracy. Interpret the results.